In [1]:
# read dataset

import pandas as pd

ratings_df       = pd.read_csv('ml-25m/ratings.csv')
movies_df        = pd.read_csv('ml-25m/movies.csv')
tags_df          = pd.read_csv('ml-25m/tags.csv')
genome_scores_df = pd.read_csv('ml-25m/genome-scores.csv')
genome_tags_df   = pd.read_csv('ml-25m/genome-tags.csv')

# ratings_df.head(5) # userId  movieId  rating   timestamp
# movies_df.head(5) # movieId  title  genres
print(ratings_df) 
# tags_df.head(5)
# genome_scores_df.head(5)
# genome_tags_df.head(5)

ModuleNotFoundError: No module named 'csv'

In [6]:
movie_w_least_5_ratings = ratings_df.groupby(['movieId']).size()
print('# movies: %d' % len(movie_w_least_5_ratings))

movie_enough_ratings_df = movie_w_least_5_ratings[movie_w_least_5_ratings >= 5].reset_index()[['movieId']]
print('# movies with at least 5 ratings: %d' % len(movie_enough_ratings_df))
print(movie_enough_ratings_df)
# need rating prediction

print(movie_enough_ratings_df['movieId'].to_numpy())


NameError: name 'ratings_df' is not defined

In [7]:
user_w_least_20_ratings = ratings_df.groupby(['userId']).size()
print('# user: %d' % len(user_w_least_20_ratings))

user_enough_ratings_df = user_w_least_20_ratings[user_w_least_20_ratings >= 20].reset_index()[['userId']]
print('# user with at least 20 ratings: %d' % len(user_enough_ratings_df))

# movie_w_least_5_ratings = ratings_df.groupby(['movieId']).size()
# print('# movies: %d' % len(movie_w_least_5_ratings))

# movie_enough_ratings_df = movie_w_least_5_ratings[movie_w_least_5_ratings >= 50].reset_index()[['movieId']]
# print('# movies with at least 5 ratings: %d' % len(movie_enough_ratings_df))
# print(movie_enough_ratings_df)
# # rating >= 100: 10326
# # rating >= 50: 13176

# # need rating prediction

# used_movies = movie_enough_ratings_df['movieId'].to_numpy()

# # df = df[~df['date'].isin(a)]
# movies_df = movies_df[movies_df['movieId'].isin(used_movies)]
# ratings_df = ratings_df[ratings_df['movieId'].isin(used_movies)]

# ratings_df

NameError: name 'ratings_df' is not defined

In [61]:
ratings_available_df = pd.merge(ratings_df, movie_enough_ratings_df)
print('# remaining ratings: %d' % len(ratings_available_df))


# remaining ratings: 24945870


In [63]:
## https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns

def weighted_average_score(df, k=0.8):
    n_views = df.groupby('movieId', sort=False).movieId.count()
    ratings = df.groupby('movieId', sort=False).rating.mean()
    scores = ((1-k)*(n_views/n_views.max()) +
             k*(ratings/ratings.max())).to_numpy().argsort()[::-1]
    df_deduped = df.groupby('movieId', sort=False).agg({'title': 'first',
                                                        'genres': 'first',
                                                        'rating': 'mean'})
    return df_deduped.assign(views=n_views).iloc[scores]

df = movies_df.merge(ratings_df)
weighted_average_score(df).head(10)


,title,genres,rating,views
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.188912,79672
356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.048011,81491
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.151342,74127
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.154099,72674
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.120189,68717
527,Schindler's List (1993),Drama|War,4.247579,60411
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.284353,55366
2959,Fight Club (1999),Action|Crime|Drama|Thriller,4.228311,58773


In [64]:
genre_popularity = (movies_df.genres.str.split('|')
                   .explode()
                   .value_counts()
                   .sort_values(ascending=False))

genre_popularity.head(10)
## no genre --> fill by tags?

Drama                 25606
Comedy                16870
Thriller               8654
Romance                7719
Action                 7348
Horror                 5989
Documentary            5605
Crime                  5319
(no genres listed)     5062
Adventure              4145
Name: genres, dtype: int64

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

movies_df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [72]:
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                                             for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tf.fit_transform(movies_df['genres'])
#tfidf_matrix.shape # (62423, 897)
print(tfidf_matrix)


  (0, 400)	0.23757373250340494
  (0, 315)	0.2526440737243718
  (0, 301)	0.2370915665629815
  (0, 298)	0.2147831517057185
  (0, 192)	0.24026120128193318
  (0, 178)	0.22949714237049806
  (0, 174)	0.22570362967006455
  (0, 163)	0.2397416749702041
  (0, 160)	0.234180144118369
  (0, 159)	0.22133722516684318
  (0, 506)	0.18995339036038658
  (0, 435)	0.19855795455376224
  (0, 396)	0.177976083190539
  (0, 351)	0.20739004333085964
  (0, 311)	0.18915070825222186
  (0, 297)	0.18302292022799407
  (0, 237)	0.19112595766467466
  (0, 188)	0.1834121977850732
  (0, 173)	0.18982544711544252
  (0, 158)	0.20179817699263547
  (0, 729)	0.14765043969407138
  (0, 471)	0.08254695445279923
  (0, 395)	0.14507520589296213
  (0, 296)	0.14514835988809247
  (0, 157)	0.13273454229156353
  :	:
  (62414, 564)	0.4334388207669775
  (62415, 0)	1.0
  (62416, 494)	0.7705044710961112
  (62416, 666)	0.4039586639256304
  (62416, 471)	0.49309254492470433
  (62417, 57)	0.5797834751838803
  (62417, 47)	0.4021920418391035
  (62417

In [75]:
# z = pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names(), index=movies_df.title).sample(25, axis=1).sample(25, axis=0)
z = pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names(), index=movies_df.title)

# print(z)
# z.to_csv('tfidf.csv', sep='\t')


                                    ((no genres listed),)  (Action,)  \
title                                                                  
Toy Story (1995)                                      0.0   0.000000   
Jumanji (1995)                                        0.0   0.000000   
Grumpier Old Men (1995)                               0.0   0.000000   
Waiting to Exhale (1995)                              0.0   0.000000   
Father of the Bride Part II (1995)                    0.0   0.000000   
...                                                   ...        ...   
We (2018)                                             0.0   0.000000   
Window of the Soul (2001)                             0.0   0.000000   
Bad Poems (2018)                                      0.0   0.000000   
A Girl Thing (2001)                                   1.0   0.000000   
Women of Devil's Island (1962)                        0.0   0.277425   

                                    (Action, Adventure)  \
titl

In [76]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

cosine_sim_df = pd.DataFrame(cosine_sim, index=movies_df['title'], columns=movies_df['title'])
print('Shape:', cosine_sim_df.shape) # Shape: (62423, 62423)
# cosine_sim_df.sample(5, axis=1).round(2)


Shape: (62423, 62423)


title,Cosmos,Casper (1995),UMMAH - Unter Freunden (2013),Recep İvedik 3 (2010),Emerald City (1988)
title,,,,,
Toy Story (1995),0.0,0.27,0.08,0.0,0.04
Jumanji (1995),0.0,0.58,0.00,0.0,0.00
Grumpier Old Men (1995),0.0,0.00,0.42,0.0,0.21
Waiting to Exhale (1995),0.0,0.00,0.25,0.0,0.51
Father of the Bride Part II (1995),0.0,0.00,1.00,0.0,0.49
...,...,...,...,...,...
We (2018),0.0,0.00,0.00,0.0,0.40
Window of the Soul (2001),0.0,0.00,0.00,0.0,0.00
Bad Poems (2018),0.0,0.00,0.49,0.0,1.00


In [79]:
def genre_recommendations(i, M, items, k=10):
    """
    Recommends movies based on a similarity dataframe

    Parameters
    ----------
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return

    """
    ix = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[ix[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

# movies_df[movies_df.title.eq('2001: A Space Odyssey (1968)')]

genre_recommendations('2001: A Space Odyssey (1968)', cosine_sim_df, movies_df[['title', 'genres']])


,title,genres
0,Enemy Mine (1985),Adventure|Drama|Sci-Fi
1,Aurora (1998),Adventure|Drama|Sci-Fi
2,Icarus (2016),Adventure|Drama|Sci-Fi
3,"20,000 Leagues Under the Sea (1954)",Adventure|Drama|Sci-Fi
4,Until the End of the World (Bis ans Ende der W...,Adventure|Drama|Sci-Fi
5,Crumbs (2015),Adventure|Drama|Sci-Fi
6,Guest from the Future (Gostya iz buduschego) (...,Adventure|Drama|Sci-Fi
7,A.I. Artificial Intelligence (2001),Adventure|Drama|Sci-Fi
8,Close Encounters of the Third Kind (1977),Adventure|Drama|Sci-Fi
9,Star Trek: Generations (1994),Adventure|Drama|Sci-Fi


In [81]:
# movies_df[movies_df.title.eq('Eternal Sunshine of the Spotless Mind (2004)')]

genre_recommendations('Eternal Sunshine of the Spotless Mind (2004)', cosine_sim_df, movies_df[['title', 'genres']])

,title,genres
0,Emmanuelle: First Contact (1994),Drama|Romance|Sci-Fi
1,The UFO Incident (1975),Drama|Romance|Sci-Fi
2,Terug Naar Morgen (2015),Drama|Romance|Sci-Fi
3,Upside Down (2012),Drama|Romance|Sci-Fi
4,11 Minutes Ago (2007),Drama|Romance|Sci-Fi
5,Dirty Computer (2018),Drama|Romance|Sci-Fi
6,"Ah of Life, The (2010)",Drama|Romance|Sci-Fi
7,Perfect Sense (2011),Drama|Romance|Sci-Fi
8,Solaris (2002),Drama|Romance|Sci-Fi
9,Her (2013),Drama|Romance|Sci-Fi


In [83]:
# movies_df[movies_df.title.eq('Love Actually (2003)')]

genre_recommendations('Love Actually (2003)', cosine_sim_df, movies_df[['title', 'genres']])

,title,genres
0,Ladies in Lavender (2004),Comedy|Drama|Romance
1,"Letzte schöne Herbsttag, Der (2010)",Comedy|Drama|Romance
2,"Arise, My Love (1940)",Comedy|Drama|Romance
3,Everything Will Be All Right (1995),Comedy|Drama|Romance
4,Kwik Stop (2001),Comedy|Drama|Romance
5,Gyeongju (2014),Comedy|Drama|Romance
6,Pyaar To Hona Hi Tha (1998),Comedy|Drama|Romance
7,The Cutting Edge 3: Chasing the Dream (2008),Comedy|Drama|Romance
8,"Same Love, Same Rain (El mismo amor, la misma ...",Comedy|Drama|Romance
9,Kadhalum Kadanthu Pogum (2016),Comedy|Drama|Romance


In [84]:
genre_recommendations('Fight Club (1999)', cosine_sim_df, movies_df[['title', 'genres']])

,title,genres
0,Cop Land (1997),Action|Crime|Drama|Thriller
1,"Corruptor, The (1999)",Action|Crime|Drama|Thriller
2,"Crew, The (2008)",Action|Crime|Drama|Thriller
3,Detour (1998),Action|Crime|Drama|Thriller
4,Jallikattu (2019),Action|Crime|Drama|Thriller
5,Breaking Point (2009),Action|Crime|Drama|Thriller
6,Rampart (2011),Action|Crime|Drama|Thriller
7,Corrupt (1999),Action|Crime|Drama|Thriller
8,"Tesseract, The (2003)",Action|Crime|Drama|Thriller
9,Saheb Biwi Aur Gangster Returns (2013),Action|Crime|Drama|Thriller


In [86]:
genre_recommendations('Jungle Book, The (1967)', cosine_sim_df, movies_df[['title', 'genres']])

,title,genres
0,Happily Ever After (1993),Animation|Children|Comedy|Musical
1,Snoopy Come Home (1972),Animation|Children|Comedy|Musical
2,Steamboat Willie (1928),Animation|Children|Comedy|Musical
3,"Claymation Christmas Celebration, A (1987)",Animation|Children|Comedy|Musical
4,Three Little Pigs (1933),Animation|Children|Comedy|Musical
5,Alvin and the Chipmunks: The Squeakquel (2009),Animation|Children|Comedy|Musical
6,Ferngully: The Last Rainforest (1992),Animation|Children|Comedy|Musical
7,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
8,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy
9,Robin Hood (1973),Adventure|Animation|Children|Comedy|Musical
